In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [81]:
# 데이터 로드 및 기본 정보
# 원본 데이터 로드
df_original = pd.read_csv('df_merged.csv')
df = df_original.copy()  # 전처리용 복사본

In [82]:
df.shape

(405, 26)

In [83]:
df.value_counts()

month  CO2ppm       Temp       Humid      VPD       Chl_a  Chl_b  TChl   Car   Chl_a_b  TCh-Car  ABS-RC  Dio-RC  Tro-RC  Eto-RC  PI_abs  DF_abs  SFI_abs  Fv-Fm  Leaf_ExtractionYield  Root_ExtractionYield  Leaf_TPC  Root_TPC  Leaf_TFC  Root_TFC  scenario
9      1225.166000  25.890000  55.446000  2.571000  2.63   0.39   3.02   1.17  6.73     2.57     2.903   0.794   2.110   0.562   0.333   -0.478  0.067    0.727  7.1                   14.6                  7.743     5.277     1.455     0.513     SSP5        1
5      371.850683   16.930256  82.488003  1.534584  9.66   2.44   12.10  3.11  3.96     3.89     1.637   0.282   1.355   0.613   2.421    0.384  0.229    0.828  20.7                  20.4                  7.369     6.396     5.242     0.841     SSP1        1
       373.935681   18.732141  81.813966  1.697796  10.53  2.58   13.11  3.37  4.08     3.90     1.629   0.273   1.355   0.539   2.012    0.304  0.203    0.832  20.7                  20.4                  7.369     6.396     5.2

### 타겟 변수와 예측 변수 분리
- 자료조사: TPC(총 페놀), TFC(총 플라보노이드)가 예측 대상 기능성 성분
- EDA 발견: TPC, TFC를 X변수로 사용하는 오류 지적
- 추출수율: 품질 지표 아님 (음의 상관관계 확인)

In [84]:
# 타겟 변수 정의
target_columns = ['Leaf_TPC', 'Root_TPC', 'Leaf_TFC', 'Root_TFC']
y = df[target_columns].copy()

In [85]:
# 예측 변수 정의 (타겟과 추출수율 제외)
exclude_columns = target_columns + ['Leaf_ExtractionYield', 'Root_ExtractionYield']
X = df.drop(columns=exclude_columns)

### 범주형 변수 처리
- 자료조사: SSP1(432ppm), SSP3(834ppm), SSP5(1089ppm) 시나리오별 특성 다름
- EDA 발견: SSP3가 일부 지표에서 SSP5보다 나쁨 (복합 스트레스)
- 시나리오는 순서형이 아닌 명목형으로 처리 필요

In [86]:
# 시나리오 변수 저장 (나중에 시나리오별 분석용)
scenario_column = X['scenario'].copy()

In [87]:
# scenario 원핫인코딩
X_encoded = pd.get_dummies(X, columns=['scenario'], prefix='scenario')

In [88]:
X_encoded.shape

(405, 22)

### 생물학적 타당성 검증 및 이상치 처리
- 자료조사: Fv/Fm 0.6 이하는 극한 스트레스 (문헌 근거)
- EDA 발견: 9월 급변은 생리적 전환 (이상치 아님)
- 에너지 보존: ABS = Tro + Dio (물리 법칙)

In [89]:
# 3-1. Fv/Fm 범위 검증 및 처리
fv_fm_extreme = (X_encoded['Fv-Fm'] < 0.6).sum()
print(f"Fv/Fm < 0.6 (극한 스트레스): {fv_fm_extreme}개")

Fv/Fm < 0.6 (극한 스트레스): 5개


In [90]:
# 극한 스트레스 플래그 추가 (제거하지 않음)
X_encoded['extreme_stress_fv'] = (X_encoded['Fv-Fm'] < 0.6).astype(int)
print("→ 처리: 제거하지 않고 플래그 변수 추가 (극한 상황 정보 보존)")

→ 처리: 제거하지 않고 플래그 변수 추가 (극한 상황 정보 보존)


In [91]:
# 3-2. 물리적 불가능 값 제거
physically_impossible = ((X_encoded['Fv-Fm'] < 0) | (X_encoded['Fv-Fm'] > 1)).sum()
print(f"Fv/Fm 범위 밖 (0-1): {physically_impossible}개")

Fv/Fm 범위 밖 (0-1): 0개


In [92]:
if physically_impossible > 0:
    before_shape = X_encoded.shape[0]
    valid_idx = (X_encoded['Fv-Fm'] >= 0) & (X_encoded['Fv-Fm'] <= 1)
    X_encoded = X_encoded[valid_idx]
    y = y[valid_idx]
    print(f"→ 처리: 물리적 불가능 값 제거 ({before_shape} → {X_encoded.shape[0]})")
else:
    print("→ 처리: 물리적 불가능 값 없음")

→ 처리: 물리적 불가능 값 없음


In [93]:
# 3-3. 에너지 보존 법칙 검증
X_encoded['energy_check'] = abs(X_encoded['ABS-RC'] - (X_encoded['Tro-RC'] + X_encoded['Dio-RC']))
energy_violation = (X_encoded['energy_check'] > 1.0).sum()
print(f"에너지 보존 위반 (오차 > 1.0): {energy_violation}개")

에너지 보존 위반 (오차 > 1.0): 0개


In [94]:
# 에너지 보존 위반 플래그
X_encoded['energy_violation'] = (X_encoded['energy_check'] > 1.0).astype(int)

### Feature Engineering - 생물학적 의미 기반
- 자료조사: 5-7월 영양생장, 8-9월 생식생장 (생장 단계별 특성)
- VPD 2.5 kPa 이상 시 미세털 보호 메커니즘 작동
- SSP3 6월 급감 현상 (CO2+온도 복합 스트레스)

In [95]:
# 4-1. 생장 단계 변수
growth_stage_map = {
    5: 'vegetative_early',
    6: 'vegetative_mid',
    7: 'vegetative_late',
    8: 'reproductive',
    9: 'senescence'
}
X_encoded['growth_stage'] = X_encoded['month'].map(growth_stage_map)
X_encoded = pd.get_dummies(X_encoded, columns=['growth_stage'], prefix='stage')

In [96]:
# 4-2. 스트레스 지표
X_encoded['vpd_stress'] = (X_encoded['VPD'] > 2.5).astype(int)
print(f"VPD > 2.5 kPa: {X_encoded['vpd_stress'].sum()}개 ({X_encoded['vpd_stress'].mean()*100:.1f}%)")

VPD > 2.5 kPa: 155개 (38.3%)


In [97]:
# 온도 스트레스 (15-25°C 적정)
X_encoded['temp_optimal'] = ((X_encoded['Temp'] >= 15) & (X_encoded['Temp'] <= 25)).astype(int)
X_encoded['heat_stress'] = (X_encoded['Temp'] > 25).astype(int)
X_encoded['cold_stress'] = (X_encoded['Temp'] < 15).astype(int)

In [98]:
# CO2 독성 (800ppm 이상)
X_encoded['co2_toxic'] = (X_encoded['CO2ppm'] > 800).astype(int)
print(f"CO2 > 800ppm (독성): {X_encoded['co2_toxic'].sum()}개")

CO2 > 800ppm (독성): 268개


In [99]:
# 4-3. 복합 스트레스 지수
# 온도-CO2 복합 스트레스
X_encoded['stress_index'] = (X_encoded['Temp'] - 20) * (X_encoded['CO2ppm'] - 432) / 432

In [100]:
# SSP3 6월 이상 플래그
X_encoded['ssp3_june_anomaly'] = ((X_encoded['scenario_SSP3'] == 1) & (X_encoded['month'] == 6)).astype(int)
print(f"SSP3 6월 이상: {X_encoded['ssp3_june_anomaly'].sum()}개")

SSP3 6월 이상: 30개


In [101]:
# 4-4. 광합성 효율 지표
# 에너지 효율 (전자전달/포획)
X_encoded['energy_efficiency'] = np.where(X_encoded['Tro-RC'] > 0, 
                                         X_encoded['Eto-RC'] / X_encoded['Tro-RC'], 
                                         0)

In [102]:
# 열소산 비율 (보호 메커니즘)
X_encoded['dissipation_ratio'] = np.where(X_encoded['ABS-RC'] > 0,
                                         X_encoded['Dio-RC'] / X_encoded['ABS-RC'],
                                         0)

In [103]:
# 4-5. 9월 생리적 전환 플래그 (이상치 x)
X_encoded['september_transition'] = (X_encoded['month'] == 9).astype(int)
print(f"9월 데이터: {X_encoded['september_transition'].sum()}개 (자료조사: 자원 재분배 시기)")

9월 데이터: 84개 (자료조사: 자원 재분배 시기)


### 변수 선택 - 다중공선성 처리
- EDA 발견: TChl = Chl_a + Chl_b (강한 상관)
- VPD는 온도의 함수 (Clausius-Clapeyron 방정식)
- 각 변수의 생물학적 의미는 다르므로 신중히 선택

In [104]:
# 상관관계 확인
numeric_cols = X_encoded.select_dtypes(include=[np.number]).columns
corr_matrix = X_encoded[numeric_cols].corr()

In [105]:
# 높은 상관관계 (> 0.95) 찾기
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.95:
            high_corr_pairs.append((corr_matrix.columns[i], 
                                  corr_matrix.columns[j], 
                                  corr_matrix.iloc[i, j]))

In [106]:
print(f"상관계수 > 0.95인 변수 쌍: {len(high_corr_pairs)}개")
for var1, var2, corr in high_corr_pairs[:5]:  # 상위 5개만 표시
    print(f"  {var1} - {var2}: {corr:.3f}")

상관계수 > 0.95인 변수 쌍: 9개
  Temp - VPD: 0.997
  Chl_a - TChl: 0.987
  ABS-RC - Tro-RC: 0.966
  Dio-RC - DF_abs: -0.965
  Dio-RC - Fv-Fm: -0.962


In [107]:
# TChl 제거 (Chl_a + Chl_b와 중복)
# EDA 발견: TChl = Chl_a + Chl_b (강한 상관)
if 'TChl' in X_encoded.columns and 'Chl_a' in X_encoded.columns and 'Chl_b' in X_encoded.columns:
    X_encoded = X_encoded.drop(columns=['TChl'])
    print("\n→ 처리: TChl 제거 (Chl_a + Chl_b와 중복)")


→ 처리: TChl 제거 (Chl_a + Chl_b와 중복)


In [108]:
# energy_check 컬럼 제거 (임시 변수)
if 'energy_check' in X_encoded.columns:
    X_encoded = X_encoded.drop(columns=['energy_check'])

### 시나리오별 상관관계 분석

In [109]:
# 시나리오별 분석을 위해 원래 시나리오 정보 복원
X_with_scenario = X_encoded.copy()
X_with_scenario['scenario'] = scenario_column.values[:len(X_encoded)]

In [110]:
scenarios = ['SSP1', 'SSP3', 'SSP5']
scenario_correlations = {}

In [115]:
for scenario in scenarios:
    scenario_mask = X_with_scenario['scenario'] == scenario
    scenario_data = X_with_scenario[scenario_mask]
    
    print(f"\n{scenario} (n={scenario_mask.sum()}):")
    
    # 각 타겟에 대한 상관관계
    for target in target_columns[:2]:  # 대표로 2개만 표시
        target_data = y[scenario_mask][target]
        
        # 상위 5개 변수
        correlations = []
        for col in numeric_cols:
            if col in scenario_data.columns and col != 'scenario':
                corr = scenario_data[col].corr(target_data)
                if not pd.isna(corr):
                    correlations.append((col, corr))
        
        correlations.sort(key=lambda x: abs(x[1]), reverse=True)
        
        print(f"  {target} 상위 상관변수:")
        for col, corr in correlations[:5]:
            print(f"    {col}: {corr:.3f}")
    
    scenario_correlations[scenario] = correlations


SSP1 (n=135):
  Leaf_TPC 상위 상관변수:
    PI_abs: 0.887
    SFI_abs: 0.886
    DF_abs: 0.867
    ABS-RC: -0.850
    Tro-RC: -0.847
  Root_TPC 상위 상관변수:
    Temp: -0.852
    VPD: -0.831
    Humid: 0.705
    month: -0.645
    Car: 0.573

SSP3 (n=135):
  Leaf_TPC 상위 상관변수:
    Eto-RC: 0.758
    september_transition: -0.725
    vpd_stress: 0.644
    energy_efficiency: 0.619
    Fv-Fm: 0.589
  Root_TPC 상위 상관변수:
    month: -0.850
    Car: 0.838
    PI_abs: 0.811
    SFI_abs: 0.809
    ABS-RC: -0.768

SSP5 (n=135):
  Leaf_TPC 상위 상관변수:
    Chl_a: -0.595
    Temp: 0.562
    stress_index: 0.546
    VPD: 0.539
    Car: -0.520
  Root_TPC 상위 상관변수:
    Temp: -0.950
    VPD: -0.945
    stress_index: -0.938
    Eto-RC: -0.722
    vpd_stress: -0.689


In [116]:
# scenario 컬럼 다시 제거
X_encoded = X_with_scenario.drop(columns=['scenario'])

### 스케일링 전략
- Tree 기반 모델: 스케일링 불필요
- Linear 모델: StandardScaler (정규분포 가정)
- Neural Network: MinMaxScaler (0-1 범위)
- 현재는 표준화만 수행 (모델 선택 후 조정 가능)

In [118]:
# 수치형 변수만 스케일링
numeric_features = X_encoded.select_dtypes(include=[np.number]).columns
categorical_features = X_encoded.select_dtypes(exclude=[np.number]).columns

print(f"수치형 변수: {len(numeric_features)}개")
print(f"범주형 변수: {len(categorical_features)}개")

수치형 변수: 30개
범주형 변수: 8개


In [119]:
# StandardScaler 적용
scaler = StandardScaler()
X_scaled = X_encoded.copy()
X_scaled[numeric_features] = scaler.fit_transform(X_encoded[numeric_features].fillna(0))

### 전처리 결과 검증

In [121]:
# 전후 비교
print(f"원본 데이터: {df_original.shape}")
print(f"최종 X shape: {X_scaled.shape}")
print(f"최종 y shape: {y.shape}")

원본 데이터: (405, 26)
최종 X shape: (405, 38)
최종 y shape: (405, 4)


In [122]:
# 새로 생성된 변수
new_features = [col for col in X_scaled.columns if col not in df_original.columns]
print(f"\n새로 생성된 변수: {len(new_features)}개")
print("주요 생성 변수:")
for feat in ['extreme_stress_fv', 'vpd_stress', 'heat_stress', 
             'stress_index', 'energy_efficiency', 'dissipation_ratio']:
    if feat in X_scaled.columns:
        print(f"  - {feat}")


새로 생성된 변수: 20개
주요 생성 변수:
  - extreme_stress_fv
  - vpd_stress
  - heat_stress
  - stress_index
  - energy_efficiency
  - dissipation_ratio


In [123]:
# 제거된 변수
removed_features = [col for col in df_original.columns 
                   if col not in X_scaled.columns and col not in target_columns + ['Leaf_ExtractionYield', 'Root_ExtractionYield']]
print(f"\n제거된 변수: {removed_features}")


제거된 변수: ['TChl', 'scenario']


### 데이터 저장

In [124]:
# 전처리된 데이터 저장
X_scaled.to_csv('X_preprocessed.csv', index=False)
y.to_csv('y_targets.csv', index=False)

In [125]:
# 시나리오 정보 저장 (모델 평가용)
scenario_info = pd.DataFrame({
    'scenario': scenario_column.values[:len(X_scaled)]
})
scenario_info.to_csv('scenario_info.csv', index=False)

print("\n저장된 파일:")
print("  - X_preprocessed.csv: 전처리된 예측 변수")
print("  - y_targets.csv: 타겟 변수")
print("  - scenario_info.csv: 시나리오 정보 (평가용)")


저장된 파일:
  - X_preprocessed.csv: 전처리된 예측 변수
  - y_targets.csv: 타겟 변수
  - scenario_info.csv: 시나리오 정보 (평가용)
